# **Movie Recommendation System**

---

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
movies_list = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [3]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044


In [4]:
movies_list.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [5]:
movies_list = movies_list.drop('genres', axis=1)
movies_list.head(1)

,movieId,title
0,1,Toy Story (1995)


In [6]:
movies_list['title'] = movies_list['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()
movies_list.head(2)

,movieId,title
0,1,Toy Story
1,2,Jumanji


In [7]:
ratings_with_movies = ratings.merge(movies_list, on='movieId')
ratings_with_movies.head(2)

,userId,movieId,rating,timestamp,title
0,1,296,5.0,1147880044,Pulp Fiction
1,3,296,5.0,1439474476,Pulp Fiction


In [8]:
ratings_with_movies.shape

(25000095, 5)

In [9]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [10]:
movies = movies[['genres', 'id', 'keywords', 'overview', 'title']]
movies.head(1)

,genres,id,keywords,overview,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar


In [11]:
movies = movies.merge(movies_list, on='title')
movies.head(1)

,genres,id,keywords,overview,title,movieId
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,72998


In [12]:
movies.shape

(4323, 6)

In [13]:
ratings_with_movies = ratings_with_movies.drop_duplicates(subset=['userId', 'movieId', 'rating'])
ratings_with_movies.shape

(25000095, 5)

In [14]:
movies_ratings_all = movies.merge(ratings, on = 'movieId')
movies_ratings_with_newID = movies_ratings_all[['userId', 'id', 'title', 'rating']]
movies_ratings_with_newID.head(1)

,userId,id,title,rating
0,3,19995,Avatar,4.0


In [15]:
movies_ratings_with_newID.shape

(12680181, 4)

In [16]:
movies_ratings_with_newID = movies_ratings_with_newID.drop_duplicates(subset=['userId', 'id', 'rating'])
movies_ratings_with_newID.shape

(12664758, 4)

#### At this Point we have only two dataframe
    1. movies ratings with changed movie ID (rating_main)
    2. movies details (movies)

In [17]:
rating_main = movies_ratings_with_newID
rating_main.head(1)

,userId,id,title,rating
0,3,19995,Avatar,4.0


In [18]:
rating_main.shape

(12664758, 4)

In [19]:
movies = movies.drop('movieId', axis=1)
movies.head(1)
movies.shape

(4323, 5)

DATA PREPARATION DONE....

In [20]:
x = rating_main.groupby('userId').count()['rating'] > 200
genuine_user = x[x].index

In [21]:
filtered_rating = rating_main[rating_main['userId'].isin(genuine_user)]
filtered_rating.shape

(5363482, 4)

In [22]:
y = filtered_rating.groupby('title').count()['rating']>=50
famous_movies = y[y].index

In [23]:
final_ratings = filtered_rating[filtered_rating['title'].isin(famous_movies)]
final_ratings.shape

(5354017, 4)

In [24]:
user_movie_rating_table = final_ratings.pivot_table(index='userId',columns='id',values='rating')

In [25]:
user_movie_rating_table.fillna(0, inplace=True)

In [26]:
user_movie_rating_table

id,5,12,13,14,16,19,20,22,24,25,...,342521,345911,347969,354110,360339,365222,374461,376659,396152,417859
userId,,,,,,,,,,,,,,,,,,,,,
3,0.0,4.0,4.0,5.0,0.0,0.00,0.0,4.0,4.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,4.0,4.0,4.0,0.0,0.00,0.0,3.5,0.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,4.0,5.0,4.0,0.0,3.00,0.0,4.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,3.0,3.0,3.0,0.0,0.00,0.0,1.5,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,0.0,4.0,5.0,5.0,0.0,0.00,0.0,4.5,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162516,3.5,4.0,4.5,5.0,4.0,2.75,0.0,4.0,4.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
162519,0.0,0.0,0.0,5.0,2.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162521,0.0,4.0,3.5,0.0,0.0,0.00,0.0,4.0,4.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# final_ratings[final_ratings['title']=='Avatar']

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
similarity_scores = cosine_similarity(user_movie_rating_table)
similarity_scores.shape

(14550, 14550)

In [30]:
from sklearn.neighbors import NearestNeighbors

In [31]:
# Define a KNN model on cosine similarity
cf_knn_model= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)


# Fitting the model on our matrix
cf_knn_model.fit(user_movie_rating_table)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [32]:
!pip install python-Levenshtein

In [33]:
from fuzzywuzzy import process

In [146]:
def movie_recommender_engine(movie_name, matrix, cf_model, n_recs):
    # Fit model on matrix
    cf_model.fit(matrix)
    # Extract input movie ID
    movie_index = process.extractOne(movie_name, movies['title'])[2]
    
    movie_id = movies['id'].iloc[movie_index]
    print(f"this is movie index {movie_id}")
    
    # Reshape the movie vector into a 2D array
    movie_vector = matrix.iloc[movie_id].values.reshape(1, -1)
    
#     # Calculate neighbour distances
    distances, indices = cf_model.kneighbors(movie_vector, n_neighbors=n_recs+1)
    movie_rec_ids = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
    
    # List to store recommendations
    # Fetch titles based on movie IDs
    cf_recs = []
    for movie_id, distance in movie_rec_ids:
        # Fetch the movie title
        movie_title = movies[movies['id'] == movie_id]['title'].values
        if len(movie_title) > 0:
            cf_recs.append({'Title': movie_title[0], 'Distance': distance})
        else:
            cf_recs.append({'Title': 'Unknown', 'Distance': distance})
    
     
    return movie_rec_ids

In [154]:
print(movie_recommender_engine('Delgo', user_movie_rating_table,cf_knn_model,5 ))

this is movie index 20542


IndexError: single positional indexer is out-of-bounds

In [150]:
recommendations = [(2657, 0.4512392293677585), (12211, 0.44644476333185334), (7097, 0.44212866436060105), (5098, 0.4409220833124181), (13642, 0.39423429558153744)]
cf_recs = []
for i in recommendations:
    print(i[0])
cf_recs

2657
12211
7097
5098
13642


[]

In [153]:
movies[movies['id']==4347]['title']

1548    Atonement
Name: title, dtype: object

In [52]:
movies.sample(5)

,genres,id,keywords,overview,title
1548,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",4347,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","As a 13-year-old, fledgling writer Briony Tall...",Atonement
3848,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",49020,"[{""id"": 966, ""name"": ""beach""}, {""id"": 1209, ""n...","15-year-old deep-thinking Welsh schoolboy, Oli...",Submarine
3415,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 35, ...",340816,"[{""id"": 1003, ""name"": ""photographer""}, {""id"": ...","Hilarity, romance, and transcendence prevail a...",Christmas Eve
1903,"[{""id"": 18, ""name"": ""Drama""}]",1246,"[{""id"": 276, ""name"": ""philadelphia""}, {""id"": 3...",When he loses a highly publicized virtual boxi...,Rocky Balboa
1265,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",20542,[],"In a divided land, it takes a rebellious boy a...",Delgo


In [94]:
movies.shape

(4323, 5)

In [121]:
movies['title'][1[0]]

<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
C:\Users\smabi\AppData\Local\Temp\ipykernel_22748\2012412577.py:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  movies['title'][1[0]]
C:\Users\smabi\AppData\Local\Temp\ipykernel_22748\2012412577.py:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  movies['title'][1[0]]
C:\Users\smabi\AppData\Local\Temp\ipykernel_22748\2012412577.py:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  movies['title'][1[0]]
C:\Users\smabi\AppData\Local\Temp\ipykernel_22748\2012412577.py:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  movies['title'][1[0]]
C:\Users\smabi\AppData\Local\Temp\ipykernel_22748\2012412577.py:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  movies['t

TypeError: 'int' object is not subscriptable